In [44]:
import pandas as pd

nodes_df = pd.read_csv('cifar_prunned_ranks.csv')
nodes_wide_df = nodes_df.pivot(index = 'filter_num',columns='class', values='prune_score')

classes = list(nodes_df['class'].unique())
classes.remove('overall')

In [45]:
def get_layer(filter_num, df = nodes_df, col = 'layer'):
    return df.loc[(df['filter_num'] == filter_num) & (df['class'] == df['class'].unique()[0]), col].item()


nodes_wide_df.reset_index(inplace=True)

nodes_wide_df['layer'] = nodes_wide_df['filter_num'].apply(get_layer)



layers = {}
for index, row in nodes_df[nodes_df['class'] == 'overall'].iterrows():
    if row['layer'] not in layers:
        layers[row['layer']] = []
    layers[row['layer']].append(row['filter_num'])
nodes_df.tail(10)
#nodes_wide_df['filter_num_by_layer'] = nodes_wide_df.apply(lambda row: get_layer(row['filter_num'], col='filter_num_by_layer'), axis = 1)

,filter_num,layer,filter_num_by_layer,prune_score,class
1794,154,3,37,0.034860,overall
1795,155,3,38,0.041014,overall
1796,156,3,39,0.074795,overall
1797,157,3,40,0.077561,overall
1798,158,3,41,0.048469,overall
1799,159,3,42,0.071964,overall
1800,160,3,43,0.111693,overall
1801,161,3,44,0.059062,overall
1802,162,3,45,0.186180,overall
1803,163,3,46,0.178743,overall


In [46]:
import numpy as np
from sklearn import manifold
from sklearn.metrics import euclidean_distances

def add_norm_col(df,classes=classes):
    norms = []
    norm = 0
    for index, row in df.iterrows():
        for label in classes:
            norm += row[label]**2
        norm = np.sqrt(norm)
        norms.append(norm)
    norms = np.array(norms)
    df['class_norm'] = norms

add_norm_col(nodes_wide_df)   
    
layer_similarities = {}
for layer in layers:
    layer_df = nodes_wide_df[nodes_wide_df['layer'] == layer]
    for label in classes:
        layer_df[label] = layer_df.apply(lambda row : row[label]/row['class_norm'], axis = 1)   
    layer_similarities[layer] = euclidean_distances(layer_df.iloc[:,1:-2])



layer_mds = {}
for layer in layer_similarities:
	print('layer: %s'%str(layer))
	mds = manifold.MDS(n_components=2, max_iter=3000, eps=1e-9, 
      random_state=2, dissimilarity="precomputed", n_jobs=1)
	pos = mds.fit(layer_similarities[layer]).embedding_
	layer_mds[layer] = pos

#print(layer_mds)





/Users/christopherhamblin/anaconda3/envs/graph_viz/lib/python3.7/site-packages/ipykernel/__main__.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/christopherhamblin/anaconda3/envs/graph_viz/lib/python3.7/site-packages/ipykernel/__main__.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/christopherhamblin/anaconda3/envs/graph_viz/lib/python3.7/site-packages/ipykernel/__main__.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[ro

layer: 0
layer: 1
layer: 2
layer: 3


In [7]:
nodes_df[nodes_df['class'] == classes[0]].head(5)

,filter_num,layer,filter_num_by_layer,prune_score,class
0,0,0,0,0.191817,frog
1,1,0,1,0.121295,frog
2,2,0,2,0.244416,frog
3,3,0,3,0.034859,frog
4,4,0,4,0.050036,frog


In [47]:
def layernum2name(layer,offset=1,title = 'layer'):
    return title+' '+str(layer+offset)

In [48]:
import numpy as np
import igraph as ig
import json
import urllib

N = len(nodes_wide_df.index)
sizes = list(nodes_wide_df['overall'])
format_sizes = []
for size in sizes:
    format_sizes.append(50*np.cbrt(size))
sizes = format_sizes

#N=len(data_dict['nodes'])


In [49]:
Edges_full = {}
for layer in layers:
    if layer+1 in layers:
        Edges_full[layer+1] = []
        for i in range(len(layers[layer])):
            for j in range(len(layers[layer+1])):
                Edges_full[layer+1].append((layers[layer][i],layers[layer+1][j],i,j))             
       
  
#print(Edges_full)

In [52]:
labels_list=(list(nodes_df[nodes_df['class'] == 'overall'].filter_num))
layers_list=(list(nodes_df[nodes_df['class'] == 'overall'].layer))
N = len(labels_list)



{0: [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44],
 1: [45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67],
 2: [68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116],
 3: [117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  149,
  150,
  151,
  152,
  153,
  154,
  155,
  1

In [53]:
target_class = 'frog'

#Node Opacity
'''
layer_colors = {0:'rgba(64,1,84,',
                1:'rgba(49,102,141,',
                2:'rgba(55,181,120,',
                3:'rgba(253,231,37,'}

layer_colors = {0:'rgba(49,102,141,',
                1:'rgba(64,1,84,',
                2:'rgba(55,181,120,',
                3:'rgba(253,231,37,',
                4:'156,50,50'}
'''

layer_colors = {0:'rgba(31,119,180', 
                1:'rgb(255,127,14',
                2:'rgb(44,160,44', 
                3:'rgb(214,39,40',
                4:'rgb(148, 103, 189', 
                5:'rgb(140, 86, 75',
                6:'rgb(227, 119, 194',
                7:'rgb(127, 127, 127',
                8:'rgb(188, 189, 34',
                9:'rgb(23, 190, 207'}


def color_scaling(x):
    return -(x-1)**4+1
    
colors_dict = {}
for layer in layers:
    colors_dict[layer] = []
    for node in colors_dict[layer]:
        alpha = color_scaling(nodes_df[nodes_df['class']==target_class].iloc[node].prune_score)
        colors_dict[layer].append(layer_colors[layer]+str(round(alpha,3))+')')

#print(colors_list)

In [54]:
node_positions

{0: {'X': [0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0],
  'Y': [-0.148163491619823,
   0.16864003589038232,
   -0.2811707511955231,
   -0.2178276350862777,
   -0.01145581660567143,
   -0.002794057911811084,
   0.18314253346027806,
   -0.03386840078842401,
   -0.04646452051839806,
   0.11370537764484584,
   -0.12586752346862157,
   -0.335769683533576,
   -0.08045779172951921,
   0.15231650325023818,
   0.022502081631893666,
   0.09641241471352938,
   0.09724091565195242,
   -0.009313863259974199,
   0.33387057410156057,
   0.15809104456690695,
   -0.011438130812390873,
   0.09336074366254518,
   0.009055715086743178,
   0.007095681879806054,
   0.1525619257276642,
   -0.0791124218135388,
   -0.15888205533612124,
   0.029056338992489442,
   0.2317775743186918,
   -0.05

In [33]:
#Node positions
layer_distance = 1   # distance in X direction each layer is separated by
node_positions = {}
layer_offset = 0
for layer in layer_mds:
    node_positions[layer] = {}
    node_positions[layer]['X'] = [] 
    node_positions[layer]['Y'] = [] 
    node_positions[layer]['Z'] = []  
    for i in range(len(layer_mds[layer])): 
        node_positions[layer]['Y'].append(layer_mds[layer][i][0])
        node_positions[layer]['Z'].append(layer_mds[layer][i][1])
        node_positions[layer]['X'].append(layer_offset)
    layer_offset+=1

print(node_positions[0])

{'X': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'Y': [-0.148163491619823, 0.16864003589038232, -0.2811707511955231, -0.2178276350862777, -0.01145581660567143, -0.002794057911811084, 0.18314253346027806, -0.03386840078842401, -0.04646452051839806, 0.11370537764484584, -0.12586752346862157, -0.335769683533576, -0.08045779172951921, 0.15231650325023818, 0.022502081631893666, 0.09641241471352938, 0.09724091565195242, -0.009313863259974199, 0.33387057410156057, 0.15809104456690695, -0.011438130812390873, 0.09336074366254518, 0.009055715086743178, 0.007095681879806054, 0.1525619257276642, -0.0791124218135388, -0.15888205533612124, 0.029056338992489442, 0.2317775743186918, -0.052866100219465265, 0.04792714394555562, 0.3802756052782087, -0.14708520012503484, -0.17057832419492663, -0.1290694966419374, -0.02770042358249475, 0.019296681612010987, 0.02660640432418872, -0.12945676444529827, -0.23419742430

In [43]:
#Edge selection
edge_weights = {}
edge_threshold = .1
Edges = {}
for layer in Edges_full:
    Edges[layer] = []
    edge_weights[layer] = []
    for i in range(len(Edges_full[layer])):
        edge_weight = nodes_df[nodes_df['class']==target_class].iloc[Edges_full[layer][i][0]].prune_score*nodes_df[nodes_df['class']==target_class].iloc[Edges_full[layer][i][1]].prune_score
        if edge_weight > edge_threshold:
            Edges[layer].append(Edges_full[layer][i])
            edge_weights[layer].append(edge_weight)
            
#print(Edges)

#Edge Positions
edge_positions = {}
for layer in Edges:
    edge_positions[layer] = {}
    edge_positions[layer]['X'] = []
    edge_positions[layer]['Y'] = []
    edge_positions[layer]['Z'] = []
    for edge in Edges[layer]:
        edge_positions[layer]['X'].append([node_positions[layer-1]['X'][edge[2]],node_positions[layer]['X'][edge[3]], None])# x-coordinates of edge ends
        edge_positions[layer]['Y'].append([node_positions[layer-1]['Y'][edge[2]],node_positions[layer]['Y'][edge[3]], None])
        edge_positions[layer]['Z'].append([node_positions[layer-1]['Z'][edge[2]],node_positions[layer]['Z'][edge[3]], None])    

print(edge_positions)


'''  
#node positions
Xn = []
Yn = []
Zn = []
layer_offset = 0
for layer in layer_mds:
    for i in range(len(layer_mds[layer])): 
        Yn.append(layer_mds[layer][i][0])
        Zn.append(layer_mds[layer][i][1])
        Xn.append(layer_offset)
    layer_offset+=1
    

#Edge Positions
Xe=[]
Ye=[]
Ze=[]
for e in Edges:
    Xe+=[Xn[e[0]],Xn[e[1]], None]# x-coordinates of edge ends
    Ye+=[Yn[e[0]],Yn[e[1]], None]
    Ze+=[Zn[e[0]],Zn[e[1]], None]
'''


{1: {'X': [[0, 1, None], [0, 1, None], [0, 1, None], [0, 1, None], [0, 1, None], [0, 1, None], [0, 1, None], [0, 1, None], [0, 1, None], [0, 1, None], [0, 1, None], [0, 1, None], [0, 1, None]], 'Y': [[-0.148163491619823, 0.05569315488771641, None], [-0.2811707511955231, 0.05569315488771641, None], [-0.002794057911811084, 0.05569315488771641, None], [0.09724091565195242, 0.05569315488771641, None], [0.33387057410156057, 0.05569315488771641, None], [-0.17057832419492663, 0.05569315488771641, None], [0.019296681612010987, 0.05569315488771641, None], [-0.2341974243068845, -0.3804349484342047, None], [-0.2341974243068845, 0.05569315488771641, None], [-0.2341974243068845, 0.1369868685024319, None], [-0.2341974243068845, 0.015334876174739796, None], [-0.07025298035200264, 0.05569315488771641, None], [-0.07025298035200264, 0.015334876174739796, None]], 'Z': [[-0.8042496400587162, 0.44491471397967214, None], [-0.11307106925435871, 0.44491471397967214, None], [-0.24639225600205192, 0.44491471397

'  \n#node positions\nXn = []\nYn = []\nZn = []\nlayer_offset = 0\nfor layer in layer_mds:\n    for i in range(len(layer_mds[layer])): \n        Yn.append(layer_mds[layer][i][0])\n        Zn.append(layer_mds[layer][i][1])\n        Xn.append(layer_offset)\n    layer_offset+=1\n    \n\n#Edge Positions\nXe=[]\nYe=[]\nZe=[]\nfor e in Edges:\n    Xe+=[Xn[e[0]],Xn[e[1]], None]# x-coordinates of edge ends\n    Ye+=[Yn[e[0]],Yn[e[1]], None]\n    Ze+=[Zn[e[0]],Zn[e[1]], None]\n'

In [64]:
#import chart_studio.plotly as py
import plotly.offline as py    #added
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)   #added

data = []

for layer in layers:
    #add nodes
    node_trace=go.Scatter3d(x=node_positions[layer]['X'],
               y=node_positions[layer]['Y'],
               z=node_positions[layer]['Z'],
               mode='markers',
               name=layernum2name(layer,title = 'nodes'),
               marker=dict(symbol='circle',
                             size=6,
                             opacity=.99,
                             color=colors_dict[layer],
                             #colorscale='Viridis',
                             #line=dict(color='rgb(50,50,50)', width=0.5)
                             ),
               text=labels_list,
               hoverinfo='text'
               )
        
    data.append(node_trace)
    
for layer in layers:        
    #add edges
    if layer not in edge_positions:  #skip first layer
        continue  
    edge_trace=go.Scatter3d(x=edge_positions[layer]['X'],
               y=edge_positions[layer]['Y'],
               z=edge_positions[layer]['Z'],
               name=layernum2name(layer,offset=0,title = 'edges'),
               mode='lines',
               line=dict(color='rgb(125,125,125)', width=1),
               hoverinfo='none'
               )
    data.append(edge_trace)


axis=dict(showbackground=False,
          showspikes=False,
          showline=False,
          zeroline=False,
          showgrid=False,
          showticklabels=False,
          title=''
          )

layout = go.Layout(
         #title="%s through Prunned Cifar10 CNN"%target_class,
         title = target_class,
         width=1000,
         height=1000,
         #showlegend=False,
         scene=dict(
             xaxis=dict(axis),
             yaxis=dict(axis),
             zaxis=dict(axis),
        ),
     margin=dict(
        t=100
    ),
    #hovermode='closest',
   )

#data = [trace2]
fig=go.Figure(data=data, layout=layout)



In [65]:
import dash
import dash_core_components as dcc
import dash_html_components as html

app = dash.Dash()
app.layout = html.Div([
    dcc.Graph(figure=fig)
])

app.run_server(port=8051)

#py.iplot(fig, filename='small_net')



 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8051/ (Press CTRL+C to quit)
127.0.0.1 - - [18/Apr/2020 18:44:05] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Apr/2020 18:44:09] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [18/Apr/2020 18:44:09] "GET /_dash-layout HTTP/1.1" 200 -
